In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, SVD, Reader, accuracy
from surprise.model_selection import train_test_split

In [2]:
ratings_data = pd.read_csv('./dataset/ratings.csv')
ratings_data.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [3]:
# book ratings range from 1 star -> 5 stars
# explicit rating
reader = Reader(rating_scale=(1, 5)) 

In [4]:
data = Dataset.load_from_df(ratings_data[['user_id', 'book_id', 'rating']], reader)

In [6]:
train_set, test_set = train_test_split(data, test_size=.25)

In [7]:
# learning rate default: lr_all = 0.005, regularization terms: reg_all = 0.02
model = SVD(random_state=0, n_factors=100, n_epochs=30, verbose=True,lr_all=0.005, reg_all=0.02)

In [8]:
model.fit(train_set)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29


In [9]:
test_predictions = model.test(test_set)
for i in range(10):
    print(test_predictions[i])

user: 8836       item: 8472       r_ui = 5.00   est = 4.21   {'was_impossible': False}
user: 47730      item: 5737       r_ui = 4.00   est = 3.32   {'was_impossible': False}
user: 45313      item: 4710       r_ui = 5.00   est = 3.65   {'was_impossible': False}
user: 34535      item: 4056       r_ui = 3.00   est = 3.82   {'was_impossible': False}
user: 19271      item: 8313       r_ui = 3.00   est = 4.15   {'was_impossible': False}
user: 42284      item: 4735       r_ui = 5.00   est = 3.51   {'was_impossible': False}
user: 14839      item: 4537       r_ui = 5.00   est = 3.75   {'was_impossible': False}
user: 38663      item: 7292       r_ui = 2.00   est = 3.25   {'was_impossible': False}
user: 27780      item: 8027       r_ui = 4.00   est = 4.44   {'was_impossible': False}
user: 8233       item: 8077       r_ui = 4.00   est = 3.85   {'was_impossible': False}


RMSE (Root Mean Squared Error), computed as:

$$\sqrt{\frac{1}{N} \sum(\hat{r_{ui}} - r_{ui})^2}$$

In [10]:
#model evaluation
rmse = accuracy.rmse(test_predictions)

RMSE: 0.8488


In [31]:
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [25, 30], 'lr_all': [0.002, 0.005]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)
print('Best RMSE score:', gs.best_score['rmse'])
print('Best parameters:', gs.best_params['rmse'])
# Best RMSE score: 0.8484454801491901
# Best parameters: {'n_epochs': 25, 'lr_all': 0.005}

Best RMSE score: 0.8484454801491901


- Best RMSE score: 0.8484454801491901
- Best parameters: {'n_epochs': 25, 'lr_all': 0.005}

In [11]:
best_model = SVD(random_state=0, n_factors=100, n_epochs=25, lr_all=0.005, verbose=True)
best_model.fit(data.build_full_trainset())

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24


In [12]:
from surprise import dump

# Save model
file_path = 'store/surprise_model.dump'
dump.dump(file_path, algo=best_model)

## Implement Surprise/SVD

The SVD model algorithm is very similar to the ALS algorithm presented in the ALS deep dive notebook. The two differences between the two approaches are:

- SVD additionally models the user and item biases (also called baselines in the litterature) from users and items.
- The optimization technique in ALS is Alternating Least Squares (hence the name), while SVD uses stochastic gradient descent.

### 1.1 The SVD model

SVD introduces two new scalar variables: the user biases $b_u$ and the item biases $b_i$. The user biases are supposed to capture the tendency of some users to rate items higher (or lower) than the average. The same goes for items: some items are usually rated higher than some others. The model is SVD is then as follows:

$$\hat r_{u,i} = \mu + b_u + b_i + q_{i}^{T}p_{u}$$

Where $\mu$ is the global average of all the ratings in the dataset. The regularised optimization problem naturally becomes:

$$ \sum(r_{u,i} - (\mu + b_u + b_i + q_{i}^{T}p_{u}))^2 +     \lambda(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2)$$

where $\lambda$ is a the regularization parameter.


### 1.2 Stochastic Gradient Descent

Stochastic Gradient Descent (SGD) is a very common algorithm for optimization where the parameters (here the biases and the factor vectors) are iteratively incremented with the negative gradients w.r.t the optimization function. The algorithm essentially performs the following steps for a given number of iterations:


$$b_u \leftarrow b_u + \gamma (e_{ui} - \lambda b_u)$$
$$b_i \leftarrow b_i + \gamma (e_{ui} - \lambda b_i)$$  
$$p_u \leftarrow p_u + \gamma (e_{ui} \cdot q_i - \lambda p_u)$$
$$q_i \leftarrow q_i + \gamma (e_{ui} \cdot p_u - \lambda q_i)$$

where $\gamma$ is the learning rate and $e_{ui} =  r_{ui} - \hat r_{u,i} = r_{u,i} - (\mu + b_u + b_i + q_{i}^{T}p_{u})$ is the error made by the model for the pair $(u, i)$.

In [16]:
class Manual_SVD:
    def __init__(self, n_factors=100, n_epochs=20, init_mean=0, init_std_dev=.1, lr_all=.005, reg_all=.02) -> None:
        self.n_factors = n_factors
        self.n_epochs = n_epochs
        self.init_mean = init_mean
        self.init_std_dev = init_std_dev
        self.lr = lr_all
        self.reg = reg_all

    def fit(self, trainset):
        self.sgd(trainset)
        return self

    def sgd(self, trainset):
        # biased users
        bu = np.zeros(trainset.n_users, dtype=np.double)
        bi = np.zeros(trainset.n_items, dtype=np.double)

        rng = np.random.RandomState(0)
        # pu: m*k 
        pu = rng.normal(self.init_mean, self.init_std_dev,
                        size=(trainset.n_users, self.n_factors))
        #pi: n*k
        qi = rng.normal(self.init_mean, self.init_std_dev,
                        size=(trainset.n_items, self.n_factors))

        lr = self.lr
        reg = self.reg
        n_factors = self.n_factors

        global_mean = trainset.global_mean

        for current_epoch in range(self.n_epochs):
            print("Processing epoch {}".format(current_epoch))
            for u, i, r in trainset.all_ratings():
                # compute current error
                dot = 0  # <q_i, p_u>
                for f in range(n_factors):
                    dot += qi[i, f] * pu[u, f]
                err = r - (global_mean + bu[u] + bi[i] + dot)

                # update biases, SGD
                bu[u] += lr * (err - reg * bu[u])
                bi[i] += lr * (err - reg * bi[i])

                # update factors
                for f in range(n_factors):
                    puf = pu[u, f]
                    qif = qi[i, f]
                    pu[u, f] += lr * (err * qif - reg * puf)
                    qi[i, f] += lr * (err * puf - reg * qif)
                    
        self.bu = np.asarray(bu)
        self.bi = np.asarray(bi)
        self.pu = np.asarray(pu)
        self.qi = np.asarray(qi)

    def predict(self, u, i):
        known_user = self.trainset.knows_user(u)
        known_item = self.trainset.knows_item(i)

        est = self.trainset.global_mean
        if known_user:
            est += self.bu[u]

        if known_item:
            est += self.bi[i]

        if known_user and known_item:
            est += np.dot(self.qi[i], self.pu[u])
        return est